In [132]:
#import quantumstatetomo as qst
import numpy as np
import qutip as qt

In [133]:
from quantumtomo import tomography

In [134]:
tomography

<module 'quantumtomo.tomography' from '/Users/lanabozanic/Characterizing SpinQ quantum computer/quantumtomo/tomography.py'>

In [135]:
counts_zero_Z = np.array([1, 0, 0.5, 0.5, 1/2, 1/2]) * 100
counts_one_Z = np.array([0, 1, 1/2, 1/2, 1/2, 1/2])
counts_plus_Z = np.array([1/2, 1/2, 1, 0, 1/2, 1/2])
counts_imag_Z = np.array([1/2, 1/2, 1/2, 1/2, 1, 0])

measurements = ["H", "V", "D", "A", "L", "R"]

In [136]:
tomo_object = tomography.QubitTomo(1)
rhoZero = tomo_object.qst_MLE(measurements, counts_zero_Z)
rhoOne = tomo_object.qst_MLE(measurements, counts_one_Z)
rhoPlus = tomo_object.qst_MLE(measurements, counts_plus_Z)
rhoImag = tomo_object.qst_MLE(measurements, counts_imag_Z)

In [150]:
rhoZero

      fun: 3.6864276834201776e-12
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 3.09288087e-07, -4.53155028e-06,  1.91689656e-06,  2.73552476e-06])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 40
      nit: 7
     njev: 8
   status: 0
  success: True
        x: array([ 5.37995498e-02, -1.40464910e-06,  3.30770719e-01,  4.70138295e-01])

In [149]:
M = rhoZero.hess_inv
M = M * np.identity(M.shape[1])  # numpy array
qt.Qobj(M)

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 9.91513028e-01  7.13732325e-03 -4.98071573e-02 -7.06461788e-02]
 [ 7.13732325e-03  3.08077919e-01  4.42153156e-04 -1.02062385e-03]
 [-4.98071573e-02  4.42153156e-04  7.00832148e-01 -4.24690034e-01]
 [-7.06461788e-02 -1.02062385e-03 -4.24690034e-01  3.97092135e-01]]

In [138]:
np.sqrt(np.diag(M))

array([0.99574747, 0.55504767, 0.83715718, 0.63015247])

In [139]:
np.sqrt(np.diag(M))

array([0.99574747, 0.55504767, 0.83715718, 0.63015247])

In [140]:
np.diag(M)

array([0.99151303, 0.30807792, 0.70083215, 0.39709213])

In [141]:
def density(minimized):
    
    T = tomography.create_t_matrix(1,2,minimized.x)
    density_matrix = 1/np.trace(np.matmul(T.H, T)) * np.matmul(T.H, T)
    return density_matrix

qt.Qobj(density(rhoZero))

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 1.00000000e+00+0.00000000e+00j -1.39384894e-06+1.98113596e-06j]
 [-1.39384894e-06-1.98113596e-06j  5.91911121e-12+0.00000000e+00j]]

In [142]:
def density_error(minimized):
    M = minimized.hess_inv
    M = M * np.identity(M.shape[1])  # numpy array
    pm = np.sqrt(np.diag(M))
    print(pm)
    
    T1 = tomography.create_t_matrix(1,2,minimized.x + pm)
    T2 = tomography.create_t_matrix(1,2,minimized.x - pm)
    err = tomography.create_t_matrix(1,2,pm)
    
    density_matrix1 = (1/np.trace(np.matmul(T1.H, T1))) * np.matmul(T1.H, T1)
    density_matrix2 = (1/np.trace(np.matmul(T2.H, T2))) * np.matmul(T2.H, T2)
    errdensity = 1/np.trace(np.matmul(err.H, err)) * np.matmul(err.H, err)
    return density_matrix1, density_matrix2, errdensity

In [143]:
rho = rhoOne

In [144]:
qt.Qobj(density(rho))

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1.08773919e-06+0.00000000e+00j 4.08689753e-07-9.92996431e-06j]
 [4.08689753e-07+9.92996431e-06j 9.99998912e-01+0.00000000e+00j]]

In [145]:
qt.Qobj(density_error(rho)[0])

[11.3348265   0.28475047  0.50823259  0.62592761]


Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.99427789+0.j         0.00337334-0.00415457j]
 [0.00337334+0.00415457j 0.00572211+0.j        ]]

In [146]:
qt.Qobj(density_error(rho)[1])

[11.3348265   0.28475047  0.50823259  0.62592761]


Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 9.99337351e-01+0.j         -1.15099184e-03+0.00141752j]
 [-1.15099184e-03-0.00141752j  6.62648944e-04+0.j        ]]

In [147]:
qt.Qobj(density_error(rho)[2])

[11.3348265   0.28475047  0.50823259  0.62592761]


Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[9.99372470e-01+0.j         1.12003770e-03-0.00137941j]
 [1.12003770e-03+0.00137941j 6.27530118e-04+0.j        ]]

In [56]:
qt.Qobj(density(rhoPlus))

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.49998798+0.00000000e+00j 0.49999789-7.56980905e-06j]
 [0.49999789+7.56980905e-06j 0.50001202+0.00000000e+00j]]

In [55]:
qt.Qobj(density(rhoImag))

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[4.99971997e-01+0.j         1.31680442e-05-0.49999367j]
 [1.31680442e-05+0.49999367j 5.00028003e-01+0.j        ]]

In [148]:
np.linalg.eigvals(rhoZero)

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [152]:
qt.Qobj(density(rhoZero))

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 1.00000000e+00+0.00000000e+00j -1.39384894e-06+1.98113596e-06j]
 [-1.39384894e-06-1.98113596e-06j  5.91911121e-12+0.00000000e+00j]]

In [153]:
np.linalg.eigvals(density(rhoZero))

array([1.00000000e+00+5.12949727e-28j, 5.13966237e-14-5.12949727e-28j])

In [ ]:
qutip.metrics